In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

location = "../dataset/clothing_data_processed.csv"
clothing_data = pd.read_csv(location)
clothing_data = clothing_data.drop(['user_id', 'review_text', 'review_summary'], axis=1)
clothing_data = clothing_data.astype({'bust_size': 'str'})
clothing_data

,fit,item_id,weight,rating,rented for,body type,category,height,size,age,bust_size,bust_type
0,fit,2260466,137.000000,10.0,vacation,hourglass,romper,68.0,M,28.0,34.0,d
1,fit,153475,132.000000,10.0,other,straight & narrow,gown,66.0,M,36.0,34.0,b
2,fit,1063761,137.391709,10.0,party,petite,shift,64.0,XS,32.0,34.0,c
3,fit,126335,135.000000,8.0,formal affair,pear,dress,65.0,S,34.0,34.0,c
4,fit,616682,145.000000,10.0,wedding,athletic,gown,69.0,M,27.0,34.0,b
...,...,...,...,...,...,...,...,...,...,...,...,...
192539,fit,2252812,140.000000,10.0,work,hourglass,jumpsuit,69.0,S,42.0,34.0,dd
192540,fit,682043,100.000000,10.0,work,petite,dress,61.0,XS,29.0,32.0,c
192541,fit,683251,135.000000,6.0,everyday,straight & narrow,dress,68.0,S,31.0,36.0,a
192542,fit,126335,165.000000,10.0,wedding,pear,dress,66.0,L,31.0,36.0,c


In [2]:
import collections

def get_profile(group_by_column, clothing_data_df):
    grouped_df = clothing_data_df.groupby([group_by_column])

    values = collections.defaultdict(list) 
    values_from_get_dummies = collections.defaultdict(list)
    for key, item in grouped_df:
        # print (key)
        # print(grouped_df.get_group(key), "\n\n")
        item_data = grouped_df.get_group(key)
        # print (item_data.columns)
        # print (item_data['weight'].mean())
        # break
        values[group_by_column].append(key)
        values['weight'].append(item_data['weight'].mean()) 
        values['rating'].append(item_data['rating'].mean()) 
        values['height'].append(item_data['height'].mean()) 
        values['age'].append(item_data['age'].mean()) 
        total_length = len(item_data) 
        for col in item_data.columns:
            if col in values:
                continue 
            total = sum(item_data[col])
            if total > 1:
                values_from_get_dummies[col].append(1)
            else:
                values_from_get_dummies[col].append(0)
      # break

    profile_data = pd.DataFrame()
    profile_data[group_by_column] = values[group_by_column]
    profile_data['weight'] = values['weight']
    profile_data['rating'] = values['rating']
    profile_data['height'] = values['height']
    profile_data['age'] = values['age']
    for col in values_from_get_dummies:
        profile_data[col] = values_from_get_dummies[col]

    return profile_data

In [3]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(handle_unknown='ignore')
X = clothing_data.copy()
X_object = X.select_dtypes('object')
ohe.fit(X_object)
codes = ohe.transform(X_object).toarray()
feature_names = ohe.get_feature_names(['fit', 'rented for', 'body type', 'category', 'size', 'bust_size', 'bust_type'])

clothing_data_for_item_profile = pd.concat([X.select_dtypes(exclude='object'), 
                                            pd.DataFrame(codes,columns=feature_names).astype(int)], axis=1)
clothing_data_for_item_profile
item_profile = get_profile('item_id', clothing_data_for_item_profile)
item_profile

,item_id,weight,rating,height,age,fit_fit,fit_large,fit_small,rented for_date,rented for_everyday,...,bust_type_b,bust_type_c,bust_type_d,bust_type_dd,bust_type_ddd,bust_type_f,bust_type_g,bust_type_h,bust_type_i,bust_type_j
0,123373,141.186610,8.801749,64.705539,35.069971,1,1,1,1,0,...,1,1,1,1,1,1,1,1,0,0
1,123793,133.531914,9.537923,64.505251,31.582264,1,1,1,1,0,...,1,1,1,1,1,1,1,0,0,0
2,124204,137.340198,9.306560,64.615481,33.561798,1,1,1,1,0,...,1,1,1,1,0,0,1,0,0,0
3,124553,140.297371,8.563895,64.862069,32.963489,1,1,1,1,1,...,1,1,1,1,1,0,0,0,0,0
4,125424,136.960791,8.995708,65.055794,32.648069,1,1,1,1,1,...,1,1,1,1,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5845,2963850,135.561375,9.818182,63.181818,30.818182,1,0,0,0,1,...,1,1,0,1,0,0,0,0,0,0
5846,2964470,149.347927,10.000000,60.250000,39.000000,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
5847,2965009,127.097927,9.500000,66.250000,32.500000,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
5848,2965924,141.333333,9.333333,65.000000,43.000000,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [4]:
def get_user_data():
#     ['fit', 'weight', 'rented for', 'body type', 'category', 'height', 'size', 'age', 'bust_size', 'bust_type']
    input_values = {}
    
    age = 0
    while age <= 0 or age > 100 :
        age = input("Enter age: (0 to 100)")
        try:
            age = int(age)
        except:
            age = 0
    input_values['age'] = age
    
    height = 0
    while height <= 0 or height > 96:
        height = input("Enter height: (in inches)")
        try:
            height = int(height)
        except:
            height = 0
    input_values['height'] = height
    
    weight = 0
    while weight <= 0 or weight > 500:
        weight = input("Enter weight: (in lbs)")
        try:
            weight = int(weight)
        except:
            weight = 0
    input_values['weight'] = weight
    
    fit_values = ['fit', 'small', 'large']
    user_fit = None
    while user_fit not in fit_values:
        user_fit = input("Select fit: (" + ",".join(fit_values) + ")")
    input_values['fit'] = user_fit
    
    rented_for_values = ['vacation', 'other', 'party', 'formal affair', 'wedding', 'date', 'everyday', 'work', 'party: cocktail']
    rented_for = None
    while rented_for not in rented_for_values:
        rented_for = input("Select rented for: (" + ",".join(rented_for_values) + ")")
    input_values['rented_for'] = rented_for
    
    body_type_values = ['hourglass', 'straight & narrow', 'petite', 'pear', 'athletic', 'full bust', 'apple']
    body_type = None
    while body_type not in body_type_values:
        body_type = input("Select body type: (" + ",".join(body_type_values) + ")")
    input_values['body_type'] = body_type
    
    category_values = ['romper', 'gown', 'shift', 'dress', 'leggings', 'top', 'jumpsuit', 'sweater', 'jacket', 'shirtdress', 'maxi', 'pants', 'shirt', 'mini', 'skirts', 'Others', 'blouse', 'suit', 'coat', 'bomber', 'blazer', 'vest', 'tank', 'frock', 'tunic', 'cardigan', 'culottes', 'print', 'sweatshirt']
    category = None
    while category not in category_values:
        category = input("Select category: (" + ",".join(category_values) + ")")
    input_values['category'] = category
    
    size_values = ['M', 'XS', 'S', 'XL', 'L']
    size = None
    while size not in size_values:
        size = input("Select size: (" + ",".join(size_values) + ")")
    input_values['size'] = size
    
    bust_size = 0
    while bust_size < 28 or bust_size > 48 or bust_size % 2 != 0 :
        bust_size = input("Select bust size: (28 to 48)")
        try:
            bust_size = int(bust_size)
        except:
            bust_size = 0
    input_values['bust_size'] = str(float(bust_size))
    
    bust_type_values = ['d', 'b', 'c', 'a', 'dd', 'ddd', 'f', 'g', 'aa', 'i', 'h', 'j']
    bust_type = None
    while bust_type not in bust_type_values:
        bust_type = input("Select bust type: (" + ",".join(bust_type_values) + ")")
    input_values['bust_type'] = bust_type
    
    return input_values
    
# user_input = get_user_data()
user_input = {
    'age': 45,
    'height': 65,
    'weight': 200,
    'fit': 'fit',
    'rented_for': 'vacation',
    'body_type': 'hourglass',
    'category': 'suit',
    'size': 'S',
    'bust_size': '30.0',
    'bust_type': 'a'
}

In [5]:
from sklearn.metrics import jaccard_score
from sklearn.metrics.pairwise import cosine_similarity

def find_jaccard_similarity(item_profile, user_profile):
    item_id = []
    score = []
    for index, row in item_profile.iterrows():
        item_id.append(row['item_id'])
        del row['item_id']
        item_values = list(row.values)
        score.append(jaccard_score(item_values, user_profile, average='micro'))
        # break

    jaccard_similarity_scores = pd.DataFrame()
    jaccard_similarity_scores['item_id'] = item_id
    jaccard_similarity_scores['jaccard_score'] = score
    return jaccard_similarity_scores



def find_cosine_similarity(item_profile, user_profile):
    item_id = []
    score = []
    for index, row in item_profile.iterrows():
        item_id.append(row['item_id'])
        del row['item_id']
        score.append(cosine_similarity([row], [user_profile])[0][0])

    cosine_similarity_scores = pd.DataFrame()
    cosine_similarity_scores['item_id'] = item_id
    cosine_similarity_scores['cosine_similarity'] = score
    return cosine_similarity_scores


In [6]:
def evalMethod1(user_input):
    # Cosine and jaccard 

    continuous_values = item_profile[['item_id', 'weight', 'height', 'age']].copy()
    discrete_values = item_profile.drop(['weight', 'rating', 'height', 'age'], axis = 1)

    # ['fit', 'rented for', 'body type', 'category', 'size', 'bust_size', 'bust_type']
    text_cols = ['fit', 'rented for', 'body type', 'category', 'size', 'bust_size', 'bust_type']
    text_values = []
    for col in text_cols:
        text_values.append(user_input[col])
    text_data = list(ohe.transform([text_values]).toarray()[0])
    jaccard_similarity_scores = find_jaccard_similarity(discrete_values, text_data)
    jaccard_similarity_scores


    # ['weight', 'height', 'age']
    numeric_cols = ['weight', 'height', 'age']
    numeric_data = []
    for col in numeric_cols:
        numeric_data.append(user_input[col])
    cosine_similarity_scores = find_cosine_similarity(continuous_values, numeric_data)
    cosine_similarity_scores
    all_scores = jaccard_similarity_scores.join(cosine_similarity_scores.set_index('item_id'), on='item_id')
    all_scores['average'] = all_scores.apply(lambda x : (x['jaccard_score'] + x['cosine_similarity']) / 2, axis = 1)
    item_ratings = item_profile[['item_id', 'rating']].copy()
    all_scores = all_scores.join(item_ratings.set_index('item_id'), on='item_id')
    all_scores = all_scores.sort_values(by=['average', 'rating'], ascending = False)

    return all_scores

In [7]:
# ['fit', 'rented for', 'body type', 'category', 'size', 'bust_size', 'bust_type']
text_cols = ['fit', 'rented_for', 'body_type', 'category', 'size', 'bust_size', 'bust_type']
text_data = []
for col in text_cols:
    text_data.append(user_input[col])
text_data = list(ohe.transform([text_data]).toarray()[0])

# ['weight', 'height', 'age']
numeric_cols = ['weight', 'height', 'age']
numeric_data = []
for col in numeric_cols:
    numeric_data.append(user_input[col])
    
user_data = numeric_data + text_data

item_data = item_profile.drop(['rating'], axis = 1)
cosine_similarity_scores = find_cosine_similarity(item_data, user_data)
cosine_similarity_scores
item_ratings = item_profile[['item_id', 'rating']].copy()
all_scores = cosine_similarity_scores.join(item_ratings.set_index('item_id'), on='item_id')
all_scores = all_scores.sort_values(by=['cosine_similarity', 'rating'], ascending = False)

all_scores

,item_id,cosine_similarity,rating
243,238164.0,0.999792,9.333333
1869,1030312.0,0.999728,8.560000
3564,1842150.0,0.999716,9.538462
1356,779946.0,0.999704,8.285714
3022,1582012.0,0.999670,9.571429
...,...,...,...
4084,2102076.0,0.969122,8.000000
4225,2170798.0,0.965940,10.000000
5751,2918861.0,0.965727,10.000000
4242,2180111.0,0.962093,4.000000


In [8]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

location = "../dataset/train.csv"
clothing_data = pd.read_csv(location)
clothing_data

,fit,user_id,item_id,weight,rating,rented for,review_text,body type,review_summary,category,height,size,age,bust_size,bust_type
0,small,379417,658706,120.000000,10.0,party,"It ran a little small, in my opinion, but the ...",straight & narrow,"Perfect, Unique Dress!",dress,65.0,M,34.0,34.0,b
1,fit,904971,182915,120.000000,10.0,party,Wore this to a Christmas party - so comfortabl...,straight & narrow,Great fit and material!,dress,64.0,S,52.0,34.0,b
2,small,163215,1979533,150.000000,10.0,other,As the temperatures climbed into the 90s yeste...,hourglass,Bright and fun for engagement pics!,dress,69.0,S,28.0,34.0,d
3,small,232640,880935,140.000000,8.0,wedding,The dress color was a wow! I found the dress ...,hourglass,This dress is a standout!,dress,68.0,M,38.0,34.0,c
4,fit,691117,180014,137.391709,10.0,party,"This dress is very flattering, and the cutouts...",athletic,So easy to wear and comfy! Paired with red acc...,dress,69.0,L,33.0,34.0,b
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154030,small,733207,2358935,128.000000,8.0,work,This was a cute top. I really liked the lace a...,athletic,Fun Top.,top,66.0,S,38.0,32.0,dd
154031,fit,873762,625911,160.000000,10.0,wedding,This dress was great! The style of the dress i...,athletic,"Classic design, with an edge.",dress,68.0,XL,27.0,36.0,b
154032,fit,14227,1515649,145.000000,8.0,party,no complaints,pear,fit like a glove,dress,65.0,M,37.0,32.0,d
154033,fit,421813,127081,170.000000,8.0,wedding,The fit of the dress is pretty true to size. I...,hourglass,Cancelled wedding. but gorgeous dress,gown,66.0,XL,34.0,36.0,d


In [9]:
clothing_data = clothing_data.drop(['user_id', 'review_text', 'review_summary'], axis=1)
clothing_data = clothing_data.astype({'bust_size': 'str'})
clothing_data

,fit,item_id,weight,rating,rented for,body type,category,height,size,age,bust_size,bust_type
0,small,658706,120.000000,10.0,party,straight & narrow,dress,65.0,M,34.0,34.0,b
1,fit,182915,120.000000,10.0,party,straight & narrow,dress,64.0,S,52.0,34.0,b
2,small,1979533,150.000000,10.0,other,hourglass,dress,69.0,S,28.0,34.0,d
3,small,880935,140.000000,8.0,wedding,hourglass,dress,68.0,M,38.0,34.0,c
4,fit,180014,137.391709,10.0,party,athletic,dress,69.0,L,33.0,34.0,b
...,...,...,...,...,...,...,...,...,...,...,...,...
154030,small,2358935,128.000000,8.0,work,athletic,top,66.0,S,38.0,32.0,dd
154031,fit,625911,160.000000,10.0,wedding,athletic,dress,68.0,XL,27.0,36.0,b
154032,fit,1515649,145.000000,8.0,party,pear,dress,65.0,M,37.0,32.0,d
154033,fit,127081,170.000000,8.0,wedding,hourglass,gown,66.0,XL,34.0,36.0,d


In [10]:
clothing_data_new = clothing_data.copy()
clothing_data_new['weight'] = clothing_data_new['weight'] // 10
clothing_data_new['age'] = clothing_data_new['age'] // 10 
clothing_data_new = clothing_data_new.astype({'weight': 'str', 'age': 'str', 'height': 'str'})
clothing_data_new

,fit,item_id,weight,rating,rented for,body type,category,height,size,age,bust_size,bust_type
0,small,658706,12.0,10.0,party,straight & narrow,dress,65.0,M,3.0,34.0,b
1,fit,182915,12.0,10.0,party,straight & narrow,dress,64.0,S,5.0,34.0,b
2,small,1979533,15.0,10.0,other,hourglass,dress,69.0,S,2.0,34.0,d
3,small,880935,14.0,8.0,wedding,hourglass,dress,68.0,M,3.0,34.0,c
4,fit,180014,13.0,10.0,party,athletic,dress,69.0,L,3.0,34.0,b
...,...,...,...,...,...,...,...,...,...,...,...,...
154030,small,2358935,12.0,8.0,work,athletic,top,66.0,S,3.0,32.0,dd
154031,fit,625911,16.0,10.0,wedding,athletic,dress,68.0,XL,2.0,36.0,b
154032,fit,1515649,14.0,8.0,party,pear,dress,65.0,M,3.0,32.0,d
154033,fit,127081,17.0,8.0,wedding,hourglass,gown,66.0,XL,3.0,36.0,d


In [11]:
from sklearn.preprocessing import OneHotEncoder
ohe_new = OneHotEncoder(handle_unknown='ignore')
X_new = clothing_data_new.copy()
X_object_new = X_new.select_dtypes('object')
ohe_new.fit(X_object_new)
codes_new = ohe_new.transform(X_object_new).toarray()
feature_names_new = ohe_new.get_feature_names(['fit', 'weight', 'rented for', 'body type', 'category', 'height', 'size', 'age', 'bust_size', 'bust_type'])
items_new = pd.concat([X_new.select_dtypes(exclude='object'), 
                        pd.DataFrame(codes_new, columns = feature_names_new).astype(int)], axis=1)

items_new

,item_id,rating,fit_fit,fit_large,fit_small,weight_10.0,weight_11.0,weight_12.0,weight_13.0,weight_14.0,...,bust_type_b,bust_type_c,bust_type_d,bust_type_dd,bust_type_ddd,bust_type_f,bust_type_g,bust_type_h,bust_type_i,bust_type_j
0,658706,10.0,0,0,1,0,0,1,0,0,...,1,0,0,0,0,0,0,0,0,0
1,182915,10.0,1,0,0,0,0,1,0,0,...,1,0,0,0,0,0,0,0,0,0
2,1979533,10.0,0,0,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3,880935,8.0,0,0,1,0,0,0,0,1,...,0,1,0,0,0,0,0,0,0,0
4,180014,10.0,1,0,0,0,0,0,1,0,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154030,2358935,8.0,0,0,1,0,0,1,0,0,...,0,0,0,1,0,0,0,0,0,0
154031,625911,10.0,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
154032,1515649,8.0,1,0,0,0,0,0,0,1,...,0,0,1,0,0,0,0,0,0,0
154033,127081,8.0,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [12]:
import collections

def get_item_profile(df):
    values_from_get_dummies = collections.defaultdict(list)
    item_id = []
    ratings = []
    grouped_items = df.groupby(['item_id'])
    for key, item in grouped_items:

        item_data = grouped_items.get_group(key)
        item_id.append(key)
        ratings.append(item['rating'].mean())
        for col in item_data.columns:
            if col == 'item_id' or col == 'rating':
                continue
            total = sum(item_data[col])
            if total > 1:
                values_from_get_dummies[col].append(1)
            else:
                values_from_get_dummies[col].append(0)


    profile_data = pd.DataFrame()
    profile_data['item_id'] = item_id
    profile_data['rating'] = ratings
    for col in values_from_get_dummies:
        profile_data[col] = values_from_get_dummies[col]
    return profile_data

In [13]:
items_profile_new = get_item_profile(items_new)
items_profile_new

,item_id,rating,fit_fit,fit_large,fit_small,weight_10.0,weight_11.0,weight_12.0,weight_13.0,weight_14.0,...,bust_type_b,bust_type_c,bust_type_d,bust_type_dd,bust_type_ddd,bust_type_f,bust_type_g,bust_type_h,bust_type_i,bust_type_j
0,123373,8.809437,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,0,0,0
1,123793,9.532890,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,0,0,0
2,124204,9.243878,1,1,1,1,1,1,1,1,...,1,1,1,1,0,0,1,0,0,0
3,124553,8.557029,1,1,1,1,1,1,1,1,...,1,1,1,1,1,0,0,0,0,0
4,125424,9.002667,1,1,1,1,1,1,1,1,...,1,1,1,1,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5757,2963850,9.750000,1,0,0,0,0,0,1,0,...,1,1,0,1,0,0,0,0,0,0
5758,2964470,10.000000,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5759,2965009,9.333333,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
5760,2965924,9.333333,0,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [14]:
item_profile_1 = items_profile_new.drop(['rating'], axis = 1)
item_profile_1

,item_id,fit_fit,fit_large,fit_small,weight_10.0,weight_11.0,weight_12.0,weight_13.0,weight_14.0,weight_15.0,...,bust_type_b,bust_type_c,bust_type_d,bust_type_dd,bust_type_ddd,bust_type_f,bust_type_g,bust_type_h,bust_type_i,bust_type_j
0,123373,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,0,0,0
1,123793,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,0,0,0
2,124204,1,1,1,1,1,1,1,1,1,...,1,1,1,1,0,0,1,0,0,0
3,124553,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,0,0,0,0,0
4,125424,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5757,2963850,1,0,0,0,0,0,1,0,0,...,1,1,0,1,0,0,0,0,0,0
5758,2964470,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5759,2965009,1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
5760,2965924,0,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [15]:
import evaluation as evalResult

def findRecommendataions(user_input):
    # ['fit', 'weight', 'rented for', 'body type', 'category', 'height', 'size', 'age', 'bust_size', 'bust_type']
    all_cols = ['fit', 'weight', 'rented for', 'body type', 'category', 'height', 'size', 'age', 'bust_size', 'bust_type']
    user_data = []
    for col in all_cols:
        user_data.append(user_input[col])
    user_data = list(ohe_new.transform([user_data]).toarray()[0])
    test_score = find_jaccard_similarity(item_profile_1, user_data)
    # test_score
    item_ratings = items_profile_new[['item_id', 'rating']].copy()
    test_score = test_score.join(item_ratings.set_index('item_id'), on='item_id')
    test_score = test_score.sort_values(by=['jaccard_score', 'rating'], ascending = False)
    return test_score


evalResult.evaluate_recommendation(evalMethod1, 10)

{'average_of_recommendations': 2671.6,
 'number_of_recommendations': 10,
 'total_test_cases': 10,
 '% of recommendations': 100.0}